<a href="https://colab.research.google.com/github/sungjuGit/COSMOS_Ju/blob/main/AutoDiff_rev3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autodiff:  Calculus  from another angle

## Dual Number Notation

Instead of D(a,b) we can write a + b ϵ, where ϵ satisfies ϵ^2=0.  (Some people like to recall imaginary numbers where an i is introduced with i^2=-1.)

Others like to think of how engineers just drop the O(ϵ^2) terms.

The four rules are

$ (a+b\epsilon) \pm (c+d\epsilon) = (a \pm c) +  (b \pm d)\epsilon$

$ (a+b\epsilon) * (c+d\epsilon) = (ac) + (bc+ad)\epsilon$

$ (a+b\epsilon) / (c+d\epsilon) = (a/c) + (bc-ad)/c^2 \epsilon $


In [1]:
struct D <: Number  # D is a function-derivative pair
    α::Float64
    β::Float64
end

In [2]:
import Base: +, -, *, /, >, convert, promote_rule
+(x::D, y::D) = D(x.α + y.α, x.β + y.β)
-(x::D, y:: D) = D(x.α - y.α, x.β - y.β)
*(x::D, y::D) = D(x.α*y.α, (x.β*y.α + x.α*y.β))
/(x::D, y::D) = D(x.α/y.α, (y.α*x.β - x.α*y.β)/y.α^2)
>(x::D, y::D) = x.α > y.α
convert(::Type{D}, x::Real) = D(x,zero(x))
promote_rule(::Type{D}, ::Type{<:Number}) = D

Base.show(io::IO,x::D) = print(io,x.α," + ",x.β," ϵ")

In [3]:
D(1,0)

1.0 + 0.0 ϵ

In [4]:
D(2,1) ^2

4.0 + 4.0 ϵ

In [5]:
ϵ = D(0,1)


0.0 + 1.0 ϵ

In [6]:
ϵ * ϵ

0.0 + 0.0 ϵ

In [7]:
ϵ^2


0.0 + 0.0 ϵ

In [8]:
1/(1+ϵ)  # Exact power series:  1-ϵ+ϵ²-ϵ³-...

1.0 + -1.0 ϵ

In [9]:
(1+ϵ)^5  ## Note this just works (we didn't train powers)!!

1.0 + 5.0 ϵ

In [10]:
(1+ϵ)^(-1)

1.0 + -1.0 ϵ

## ...and now the derivative of an arbitrary function defined as a computer code algorithm, almost by magic

We would like to use a simple example, computation of sqrt(x), where  how autodiff works came as both a mathematical surprise, and a computing wonder.  The example is the Babylonian algorithm, known to man for millenia, to compute sqrt(x):  


 > Repeat $ t \leftarrow  (t+x/t) / 2 $ until $t$ converges to $\sqrt{x}$.

 Each iteration has one add and two divides. For illustration purposes, 10 iterations suffice.

In [11]:
function Babylonian(x; N = 10)
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2  end
    t
end

Babylonian (generic function with 1 method)

In [12]:
x=2;

Babylonian(x),√x  # Type \sqrt+<tab> to get the symbol

(1.414213562373095, 1.4142135623730951)

In [13]:
x=49; Babylonian(D(x,1)), (√x,.5/√x)

(7.0 + 0.07142857142857142 ϵ, (7.0, 0.07142857142857142))